In [12]:
import pandas as pd
from typing import List, Any, Tuple

In [3]:
csvfile = "/Users/alifabdullah/Collaboration/Kaggle-ML-Algorithm-Musings/datasets/car_evaluation.csv"
csv_in_panda_form = pd.read_csv(csvfile)
target_column = "Decision"
feature_columns = [feature_header for feature_header in csv_in_panda_form.columns.drop(target_column)]

#print(csv_in_panda_form)
#print(feature_columns)

#for row in csv_in_panda_form.iterrows():
#  print(row[1]["Outcome"])

print(len(csv_in_panda_form[csv_in_panda_form["NumberOfDoors"] == '2']))
print((csv_in_panda_form["Decision"].unique()))
print(csv_in_panda_form.iloc[0])
  


432
['unacc' 'acc' 'vgood' 'good']
BuyingPrice        vhigh
MaintenanceCost    vhigh
NumberOfDoors          2
NumberOfPersons        2
LugBoot            small
Safety               low
Decision           unacc
Name: 0, dtype: object


In [ ]:
class Branch:
  """
  This class represents a branch from a Decision tree.
  """
  
  def __init__(self):
    self.value = None
    self.next_node : 'DecisionTreeNode' = None

  def change_value(self, value):
    self.value = value

  def change_next_node(self, next_node: 'DecisionTreeNode'):
    self.next_node = next_node

class DecisionTreeNode:
  """
  This class represents the nodes that will make up my decision tree.
  There are two types of nodes. Branching nodes - whose children are either
  more branching nodes or leaf nodes - and leaf nodes - where, upon reaching these
  nodes, you end up with a label for the test instance.
  """

  def __init__(self):
    self.list_of_branches : List[Branch] = [] # Branch Node only
    self.label = None # Leaf Node only
    self.feature_name = None # Branch Node only
    self.decision = None # Leaf Node only
  
  # Creates a Branch Node
  def add_branch(self, branch: Branch):
    self.list_of_branches.append(branch)
  
  # Creates a Leaf Node
  def change_label(self, label):
    self.label = label
  
  # Changes the feature name of a Branch Node
  def assign_feature_name(self, feature_name):
    # Ensure immutability for feature name once we assign it
    assert self.feature_name is None, "Cannot reassign feature name."

    self.feature_name = feature_name

In [25]:
def _decision_tree_maker(dataframe : pd.DataFrame, list_of_labels, features_to_consider, target_column):
  head_node = DecisionTreeNode()

  # Base Case 1: If our dataset is empty or the error is 0, meaning all labels are the same
  if len(dataframe) == 0:
    head_node.change_label(list_of_labels[0]) # There is no data to make a label with, so just pick one.
    return head_node
  
  # Base Case 2: The minority label has instance count 0, meaning all labels are the same in this dataset
  if _dataset_error_calculator(dataframe, list_of_labels, target_column) == 0:

    head_node.change_label(dataframe[target_column])
    return head_node
  
  # Induction step
  else:
    biggest_error_diff = 0
    feature_with_biggest_error_diff = None

    # Compare error before any feature split with each feature split's error:
    error_before_split = _dataset_error_calculator(dataframe, list_of_labels, target_column)
    print(f"Before feature split: {error_before_split}")

    # Loop over every feature to get its error on the dataframe
    for feature in features_to_consider:
      feature_error = _dataset_error_calculator(dataframe, list_of_labels, target_column, feature)
      print(f"{feature}: {feature_error}")

      # Find the feature split that causes the biggest error difference from before to after the split
      if error_before_split - feature_error > biggest_error_diff:
        biggest_error_diff = error_before_split - feature_error
        feature_with_biggest_error_diff = feature
    
    # Assign the current node to have the feature that causes the biggest error 
    # difference from before to after the split
    head_node.assign_feature_name(feature_with_biggest_error_diff)
    features_left = [feature_left for feature_left in features_to_consider if feature_left != feature_with_biggest_error_diff]
    print(f"All features before split: {features_to_consider}")
    list_of_unique_value_datasets, list_of_unique_values = _dataset_splitter_using_feature(dataframe, feature_with_biggest_error_diff)
    print(f"Features left: {features_left}")
    for unique_value_dataset, unique_value in zip(list_of_unique_value_datasets, list_of_unique_values):

      # Fixing how we add branches to our decision tree implementation.
      current_branch = Branch()
      current_branch.change_value(unique_value)
      next_decision_tree_node = _decision_tree_maker(unique_value_dataset, list_of_labels, features_left, target_column)
      current_branch.change_next_node(next_decision_tree_node)

      head_node.add_branch(current_branch)
    return head_node
      
    



# DEPRECATING THIS FUNCTION
def _decision_tree_split_decider(dataframe : pd.DataFrame, list_of_labels, target_column):
  """
  Takes in a Pandas dataframe, goes through every feature, figures out which 
  one gives the greatest error difference before and after the particular feature,
  chooses that to split the data, and repeats the process for the remaining
  features.
  """

  # Get all features beside target column
  all_features = dataframe.drop(columns=[target_column]).columns.to_list()

  # All non terminating nodes of the tree (that are at the end of value branches
  # a previously chosen feature).
  frontier = []

  # Repeat the decision tree construction process until all features have been looked at
  while len(all_features) > 0:

    biggest_error_diff = 0
    feature_with_biggest_error_diff = None

    # Compare error before any feature split with each feature split's error:
    error_before_split = _dataset_error_calculator(dataframe, list_of_labels, target_column)
    #print(f"Before feature split: {error_before_split}")

    # Loop over every feature to get its error on the dataframe
    for feature in all_features:
      feature_error = _dataset_error_calculator(dataframe, list_of_labels, target_column, feature)
      #print(f"{feature}: {feature_error}")

      # Find the feature split that causes the biggest error difference from before to after the split
      if error_before_split - feature_error > biggest_error_diff:
        biggest_error_diff = error_before_split - feature_error
        feature_with_biggest_error_diff = feature
    
    current_tree_node = DecisionTreeNode()
    unique_values_of_the_current_biggest_feature = dataframe[feature_with_biggest_error_diff].unique()
    print(f"Feature with biggest error diff: {feature_with_biggest_error_diff}")
    print(f"Unique Values of the feature with biggest error diff: {unique_values_of_the_current_biggest_feature}")

    # Create our tree head, and give it the feature with the biggest error difference
    # from before the split to after the split
    current_tree_node.assign_feature_name(feature_with_biggest_error_diff)

    for unique_value_of_the_current_biggest_feature in unique_values_of_the_current_biggest_feature:
      print(f"Individual unique values: {unique_value_of_the_current_biggest_feature}")
      branch_of_the_current_tree_node = Branch()
      branch_of_the_current_tree_node.change_value(unique_value_of_the_current_biggest_feature)
      current_tree_node.add_branch(branch_of_the_current_tree_node)
      print(f"Dataset splits for the feature with greatest error diff: {_dataset_splitter_using_feature(dataframe, feature_with_biggest_error_diff)}")
      
      # Get sub datasets whose values correspond to the current chosen value, and
      # the feature splitting.
    print(f"Current tree node: {current_tree_node}")
    print(f"Branches of the current tree node: {current_tree_node.list_of_branches}")
    break

def _dataset_splitter_using_feature(dataframe, feature) -> Tuple[List[Any], List[Any]]:
  """
  Takes in a feature, and then returns a list of dataset splits, one for each unique
  value of the feature, where each split has the same value for the column 'feature'
  """
  list_of_dataset_splits = []
  unique_values = dataframe[feature].unique()
  for unique_value in unique_values:
    list_of_dataset_splits.append(dataframe[dataframe[feature]==unique_value])
  return list_of_dataset_splits, unique_values

def _dataset_error_calculator(dataframe, list_of_labels, target_column, feature=None):
  """
  If you pass in a feature, get the error difference between the dataset before 
  the feature split, and after the feature split.
  """
  if feature is not None:
    unique_feature_values = dataframe[feature].unique()
    error_sum = 0
    
    # Go through every unique value of the chosen feature
    for feature_value in unique_feature_values:

      # Get all rows where the current feature value is present for the feature column.
      # Aggregate the sum of the errors in error_sum
      current_df = dataframe[dataframe[feature] == feature_value]
      current_df_error, label_to_count_mapping_dictionary_current_df = _dataset_consistency_metrics(current_df, list_of_labels, target_column)
      error_sum += current_df_error

    return error_sum
  else:

    # After looking over every label, return the label that has the least amount of
    # instances associated with it.
    error, label_to_count_mapping_dictionary = _dataset_consistency_metrics(dataframe, list_of_labels, target_column)
    return error
    

def _dataset_consistency_metrics(dataframe, list_of_labels, target_column):
  """
  Tells you the counts of instances, in a dataset, corresponding to labels, as 
  well as the error of the dataset, where the error is calculated as the number
  of dataset instances corresponding to the label with the least instances.

  Args:
    dataframe - A Pandas DataFrame, representing our dataset
    list_of_labels - List of labels that represents all labels that could be assigned (replaced by self.list_of_labels going forward)
    target_column: The name of the column we are trying to predict (replaced by self.target_column going forward)
  
  Returns:
    error - The error, as defined above
    label_counts - A dictionary mapping between labels and their respective counts
  """
  label_count_mapping_dict = dict()
  for key in list_of_labels:
    label_count_mapping_dict[key] = 0
  
  for row in dataframe.iterrows():
    label_count_mapping_dict[row[1][target_column]] += 1 
  
  error = float("inf")
  for key in list_of_labels:
    if label_count_mapping_dict[key] < error:
      error = label_count_mapping_dict[key]
  return error, label_count_mapping_dict

# print(_dataset_error_calculator(csv_in_panda_form, ['unacc','acc','vgood','good'], "Decision", None))
# print(_dataset_error_calculator(csv_in_panda_form, ['unacc','acc','vgood','good'],"Decision", "NumberOfDoors"))

# print(_decision_tree_split_decider(csv_in_panda_form, ['unacc','acc','vgood','good'], "Decision"))

all_features = csv_in_panda_form.drop(columns=["Decision"]).columns.to_list()
constructed_decision_tree = _decision_tree_maker(csv_in_panda_form, ['unacc','acc','vgood','good'], all_features, "Decision")

def _print_all_tree_parts(tree_node : DecisionTreeNode):
  print("huh",tree_node.label)
  print("what",tree_node.feature_name)
  for branch in tree_node.list_of_branches:
    #print(branch.value)
    _print_all_tree_parts(branch.next_node)


_print_all_tree_parts(constructed_decision_tree)

Before feature split: 65
BuyingPrice: 62
MaintenanceCost: 49
NumberOfDoors: 61
NumberOfPersons: 63
LugBoot: 48
Safety: 30
All features before split: ['BuyingPrice', 'MaintenanceCost', 'NumberOfDoors', 'NumberOfPersons', 'LugBoot', 'Safety']
Features left: ['BuyingPrice', 'MaintenanceCost', 'NumberOfDoors', 'NumberOfPersons', 'LugBoot']
Before feature split: 30
BuyingPrice: 30
MaintenanceCost: 30
NumberOfDoors: 30
NumberOfPersons: 30
LugBoot: 9
All features before split: ['BuyingPrice', 'MaintenanceCost', 'NumberOfDoors', 'NumberOfPersons', 'LugBoot']
Features left: ['BuyingPrice', 'MaintenanceCost', 'NumberOfDoors', 'NumberOfPersons']
Before feature split: 9
BuyingPrice: 9
MaintenanceCost: 9
NumberOfDoors: 3
NumberOfPersons: 9
All features before split: ['BuyingPrice', 'MaintenanceCost', 'NumberOfDoors', 'NumberOfPersons']
Features left: ['BuyingPrice', 'MaintenanceCost', 'NumberOfPersons']
Before feature split: 3
BuyingPrice: 3
MaintenanceCost: 3
NumberOfPersons: 0
All features before